# Creating of a Flask application showing Citibike stations


In this segment we will create a basic app, where we will connect to the Citibike database, and display the list of stations. 

Let's remember first how to get data from the database.

In [ ]:
!apt-get install python3-mysqldb
!pip install sqlalchemy sql_magic
!pip install flask==0.12.2
!pip install flask-ngrok

In [ ]:
# This code creates a connection to the database
from sqlalchemy import create_engine

conn_string = 'mysql://{user}:{password}@{host}/{db}?charset={encoding}'.format(
    host = 'db.ipeirotis.org', 
    user = 'student',
    db = 'citibike_fall2017',
    password = 'dwdstudent2015',
    encoding = 'utf8mb4')

engine = create_engine(conn_string)
con = engine.connect()

In [ ]:
# cur = con.cursor(mdb.cursors.DictCursor)
stations = con.execute("SELECT DISTINCT id, name, capacity, lat, lon  FROM status_fall2017 LIMIT 5")

for s in stations:
    print(s)
    print("-----")

## Displaying the database results in HTML

Now, we are going to display these data in HTML page in a table format. An HTML table is defined with the [`<table>` tag](https://www.w3schools.com/html/html_tables.asp). Each table row is defined with the `<tr>` tag. A table header is defined with the `<th>` tag. By default, table headings are bold and centered. A table data/cell is defined with the `<td>` tag. 


In [ ]:
# This code connects Google Drive to Colab, and makes the 
# contents of Google Drive available under "/content/drive/My Drive"
from google.colab import drive
drive.mount('/content/drive')

and add the following code to the `"templates/citibike.html"` file




```html
<!DOCTYPE html>
<html>
<body>   <!-- This is a comment in HTML -->
    <table>
        <thead>
            <tr>    <!-- Construct table header -->
                <th>Station ID</th>
                <th>Address</th>
                <th>Capacity</th>
                <th>Lon / Lat</th>
            </tr>
        </thead>
        <tbody>
            {% for station in stations %}    <!-- Fill table rows for each station --> 
                <tr>
                    <td>{{ station.station_id }}</td>
                    <td>{{ station.name }}</td>
                    <td>{{ station.capacity }}</td>
                    <td>{{ station.lon }}, {{ station.lat }}</td>
           
                </tr>
            {% endfor %}
         </tbody>
    </table>
</body>
</html>
```


#### Now let's start our server:

Notice that we moved the code for connecting to the database inside our web server code. Then, we pass the `stations` variable as a parameter to the templace HTML code.

Make sure that the following are in place:
* You have connected to your Google Drive, and have created the `citibike.html` inside the templace folder in Google Drive.
* When you connect to the webserver URL, you add the `/citibike/` suffix.

In [ ]:
from flask import Flask, render_template
from flask_ngrok import run_with_ngrok
from sqlalchemy import create_engine
 
# The code below assumes that you use Google Colab and that you have
# a "templates" folder in your Google Drive. If not, change the location
# of the template_folder accordingly.
app = Flask(__name__, template_folder = '/content/drive/My Drive/templates')
run_with_ngrok(app)
 
@app.route('/citibike')
def citibike_stations():
 
    conn_string = 'mysql://{user}:{password}@{host}/{db}?charset={encoding}'.format(
        host = 'db.ipeirotis.org', 
        user = 'student',
        db = 'citibike_fall2017',
        password = 'dwdstudent2015',
        encoding = 'utf8mb4')
 
    engine = create_engine(conn_string)
    con = engine.connect()
    stations = con.execute("SELECT DISTINCT id AS station_id, name, capacity, lat, lon  FROM status_fall2017")
    con.close()
 
    return render_template('citibike.html', stations=stations)
 
app.run()